In [2]:
from collections import Counter,defaultdict
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from pprint import pprint
import json
from bs4 import BeautifulSoup
import sys
import os
import ast
from gensim.models.doc2vec import Doc2Vec,LabeledSentence

#os.environ['KERAS_BACKEND']='theano'
import keras
import keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
import keras.objectives


from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report,jaccard_similarity_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.naive_bayes import MultinomialNB

import re

import warnings
import numpy as np
from sklearn.externals.joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.multiclass import _ConstantPredictor, OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from sklearn.linear_model import SGDClassifier

In [3]:
texts = []
labels = []
urls = []

with open('parlementData_2001-2017.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='&',quotechar='|')
    for row in spamreader:
        urls.append(row[0])
        texts.append(row[1])
        labels.append(ast.literal_eval(row[2]))
        
munTexts = []
munLabels = []
munNumber = []

with open('municipality.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='&',quotechar='|')
    for row in spamreader:
        munNumber.append(row[0])
        munTexts.append(row[1])
        munLabels.append(ast.literal_eval(row[2]))
        
with open('municipalityFam.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='&',quotechar='|')
    for row in spamreader:
        munNumber.append(row[0])
        munTexts.append(row[1])
        munLabels.append(ast.literal_eval(row[2]))

In [4]:
print(len(munLabels))

439


In [5]:
TEST_SET = 0.15
nb_test_samples = int(TEST_SET * len(labels))

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 160

lb = MultiLabelBinarizer()
labelsCNN = lb.fit_transform(labels)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labelsCNN.shape)

x_train = data[:-nb_test_samples]
y_train = labelsCNN[:-nb_test_samples]
x_test = data[-nb_test_samples:]
y_test = labelsCNN[-nb_test_samples:]

xTrain = texts[:-nb_test_samples]
xTest = texts[-nb_test_samples:]

#transformer = TfidfVectorizer(smooth_idf=False, min_df=0.00001, max_df=0.2, sublinear_tf=True)
    
#xTrain = transformer.fit_transform(xTrain)
#xTest = transformer.transform(xTest)

print('Class distribution in training and validation set ')
print(y_train.sum(axis=0))
print(y_test.sum(axis=0))

#xMun = transformer.transform(munTexts)
#yMun = lb.transform(munLabels)

Found 392519 unique tokens.
Shape of data tensor: (52397, 1000)
Shape of label tensor: (52397, 17)
Class distribution in training and validation set 
[2916 3559 6695 3454 1079 6047 1815 3192 4703 2935 8692 8222 1637 1591
 3651 2902 9145]
[ 523  632 1206  591  181 1070  330  513  873  574 1557 1437  257  294
  698  519 1595]


In [6]:
def evaluation(yVal,yPred):
    print("F1 is "        + str(round(f1_score(yVal, yPred, average="micro"),3)) + 
        ", Precision is " + str(round(precision_score(yVal, yPred, average="micro"),3)) +
        ", Recall is "    + str(round(recall_score(yVal, yPred, average="micro"),3)) +
        ", Accuracy is "  + str(round(accuracy_score(yVal, yPred),3)))


def evaluationWithBestThreshold(clf, xMun, yMun, Thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]):
    bestF1 = 0
    bestThreshold = 0
    yPred = clf.predict_proba(xMun)
    for threshold in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        F1 = f1_score(yMun, (yPred > threshold).astype(int), average="micro")
        if F1 > bestF1: 
            bestF1 = F1
            bestThreshold = threshold
    evaluation(yMun,(yPred > bestThreshold).astype(int))

In [7]:
def preprocess(article):
    article = article.encode('utf-8')
    article = str(article.lower())
    return removeTagsInterpuntion(article)

def removeTagsInterpuntion(article): #remove URL's, HTML-tags and interpuntion
    article = re.sub("<\w*>", '', article)
    article = re.sub("<\w*\s\/>", '', article)
    article = re.sub("^https?:\/\/.*[\r\n]*", '', article)
    article = re.sub('\\\\x\S.', '', article)
    article = re.sub('[^a-z\s]', '', article)
    return article

In [8]:
municipality = []

with open(r"C:\Users\PC-Axel\Documents\GitHub\thesis\Code\Data preparation\alleGemeentes2.csv", 'r') as csvfile:
    next(csvfile)
    spamreader = csv.reader(csvfile, delimiter='|')
    
    for row in spamreader:
        municipality.append(preprocess(row[1]))

indices = np.arange(len(municipality))
np.random.shuffle(indices)
municipality = np.array(municipality)[indices]

In [9]:
label = [0] * len(xTrain[:10000])
label.extend([1] * len(municipality[:10000]))
print(len(label))
allData = [] 
allData.extend(xTrain[:10000])
allData.extend(municipality[:10000])
print(len(allData))

indices = np.arange(len(label))
np.random.shuffle(indices)
allData = np.array(allData)[indices]
label = np.array(label)[indices]

xTrainWeigth = allData[:-1000]
yTrainWeigth = label[:-1000]
xTestWeigth = allData[-1000:]
yTestWeigth = label[-1000:]

transformer = TfidfVectorizer(smooth_idf=False, min_df=0.00001, max_df=0.2, sublinear_tf=True)
    
xTrainWeigth = transformer.fit_transform(xTrainWeigth)
xTestWeigth = transformer.transform(xTestWeigth)

20000
20000


In [10]:
clf =LogisticRegression(dual=False)
clf.fit(xTrainWeigth, yTrainWeigth)
yPred = clf.predict(xTestWeigth)
evaluation(yTestWeigth,yPred)

F1 is 0.998, Precision is 0.998, Recall is 0.998, Accuracy is 0.998


In [11]:
#weights = transformer.transform(xTrain[1000:])
weights = (1.-np.array(clf.predict_proba(transformer.transform(xTrain[10000:])))[:,0])/ np.array(clf.predict_proba(transformer.transform(xTrain[10000:])))[:,0]

In [12]:
print(x_train.shape)

(44538, 1000)


In [13]:
print(len(weights))

34538


In [25]:
newTransformer = TfidfVectorizer(smooth_idf=False, min_df=0.00001, max_df=0.2, sublinear_tf=True)
xTrain = newTransformer.fit_transform(xTrain[10000:])
xTest = newTransformer.transform(xTest)

In [8]:
def _fit_binary_new(estimator, X, y, sample_weight, classes=None):
    unique_y = np.unique(y)
    if len(unique_y) == 1:
        if classes is not None:
            if y[0] == -1:
                c = 0
            else:
                c = y[0]
            warnings.warn("Label %s is present in all training examples." %
                          str(classes[c]))
        estimator = _ConstantPredictor().fit(X, unique_y)
    else:
        estimator = clone(estimator)

        # Only this changed
        estimator.fit(X, y, sample_weight=sample_weight)
    return estimator

class OneVsRestClassifierNew(OneVsRestClassifier):

    def fit(self, X, y, sample_weight=None):
        self.label_binarizer_ = LabelBinarizer(sparse_output=True)
        Y = self.label_binarizer_.fit_transform(y)
        Y = Y.tocsc()
        self.classes_ = self.label_binarizer_.classes_
        columns = (col.toarray().ravel() for col in Y.T)
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(delayed(_fit_binary_new)(
            self.estimator, X, column, sample_weight, classes=[
                "not %s" % self.label_binarizer_.classes_[i],
                self.label_binarizer_.classes_[i]])
            for i, column in enumerate(columns))

        return self

In [33]:
#y_mlb = MultiLabelBinarizer().fit_transform(y_train)
clf = OneVsRestClassifierNew(SGDClassifier(alpha = 1e-05, loss = 'modified_huber', penalty = 'l2', random_state=42, max_iter=10, tol=None))
clf.fit(xTrain, y_train[10000:], sample_weight=weights)

OneVsRestClassifierNew(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', max_iter=10,
       n_iter=None, n_jobs=1, penalty='l2', power_t=0.5, random_state=42,
       shuffle=True, tol=None, verbose=0, warm_start=False),
            n_jobs=1)

In [34]:
#y_test = MultiLabelBinarizer().fit_transform(y_test)
prediction = clf.predict(xTest)
evaluationWithBestThreshold(clf, xTest, y_test)

F1 is 0.729, Precision is 0.697, Recall is 0.763, Accuracy is 0.401


In [35]:
xMun = newTransformer.transform(munTexts)
yMun = lb.transform(munLabels)

In [36]:
#prediction = clf.predict(xMun)
evaluationWithBestThreshold(clf, xMun, yMun, Thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
#evaluation(yMun,prediction)

F1 is 0.363, Precision is 0.352, Recall is 0.374, Accuracy is 0.132


In [18]:
print(len(weights))
print(x_train[10000:].shape)

34538
(34538, 1000)


In [14]:
import keras.backend as K

def fmeasure(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))

    p = true_positives / (predicted_positives + K.epsilon())
    r = true_positives / (possible_positives + K.epsilon())

    beta = 1 # fmeasure
    bb = beta**2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

import keras.objectives
keras.objectives.fmeasure = fmeasure

In [15]:
GLOVE_DIRS   = [r"C:\Users\PC-Axel\Documents\Codeer projecten\Word2Vec Vectoren\Nederlandse word2vec\combined-160.txt",
                r"C:\Users\PC-Axel\Documents\GitHub\thesis\Code\Data preparation\zelfgemaakte-w2v.txt"]
TRAINABLE    = [True, False]
SPLIT        = [True, False]
MULTIFILTER  = [True, False]
FILTERS      = [3,5,7,11]
MULTIFILTERS = [[2,3,4], [3,4,5], [7,8,9]]

In [16]:
def getEmbedding(GLOVE_DIR):
    embeddings_index = {}
    f = open(GLOVE_DIR, encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [17]:
def testVersionCNN(embeddings_index, RETRAIN, FILTER, FILTERSIZE,filepath):
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=RETRAIN)
    
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    
    if FILTER:
        convs = []
        for fsz in FILTERSIZE:
            l_conv = Conv1D(filters=128,kernel_size=fsz,activation='relu')(embedded_sequences)
            l_pool = MaxPooling1D(5)(l_conv)
            convs.append(l_pool)
        l_previous = Merge(mode='concat', concat_axis=1)(convs)
    else:
        l_cov1 = Conv1D(128, FILTERSIZE, activation='relu')(embedded_sequences)
        l_previous = MaxPooling1D(5)(l_cov1)
    
    l_cov2 = Conv1D(128, 5, activation='relu')(l_previous)
    l_pool2 = MaxPooling1D(5)(l_cov2)
    l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
    if FILTER:
        l_pool3 = MaxPooling1D(int(l_cov3.shape[1]))(l_cov3)  # global max pooling
    else:
        l_pool3 = MaxPooling1D(int(l_cov3.shape[1]))(l_cov3)
    l_flat = Flatten()(l_pool3)
    l_drop = Dropout(0.5)(l_flat)
    l_dense = Dense(128, activation='relu')(l_drop)
    preds = Dense(labelsCNN.shape[1], activation='sigmoid')(l_dense)

    modelCNN = Model(sequence_input, preds)
    modelCNN.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=[keras.metrics.categorical_accuracy, fmeasure])
    filepath= "best_models\CNN_" + filepath
    checkpoint = ModelCheckpoint(filepath, monitor="val_fmeasure", verbose=0, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    return modelCNN, callbacks_list

In [18]:
# Without split
for RETRAIN,b in zip(TRAINABLE,["static_"]):
    for GLOVE_DIR,a in zip(GLOVE_DIRS,["W2V_loaded_","W2V_own_"]):
        embeddings_index = getEmbedding(GLOVE_DIR)
        for FILTER,c in zip(MULTIFILTER,["multi_","single_"]):
            if FILTER:
                for FILTERSIZE,d in zip(MULTIFILTERS,["234","345","789"]):
                    modelCNN, callbacks_list = testVersionCNN(embeddings_index, RETRAIN, FILTER, FILTERSIZE, a+b+c+d)
                    modelCNN.fit(x_train, y_train, validation_data=(x_test, y_test),
                                 epochs=7, batch_size=128, callbacks=callbacks_list)
            else:
                for FILTERSIZE,d in zip(FILTERS,["3","5","7","11"]):
                    modelCNN, callbacks_list = testVersionCNN(embeddings_index, RETRAIN, FILTER, FILTERSIZE, a+b+c+d)
                    modelCNN.fit(x_train, y_train, validation_data=(x_test, y_test),
                                 epochs=7, batch_size=128, callbacks=callbacks_list)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 44538 samples, validate on 7859 samples
Epoch 1/7
44538/44538 [==============================] - 1268s 28ms/step - loss: 0.2830 - categorical_accuracy: 0.2076 - fmeasure: 0.1383 - val_loss: 0.2215 - val_categorical_accuracy: 0.3771 - val_fmeasure: 0.4027
Epoch 2/7
20224/44538 [============>.................] - ETA: 10:00 - loss: 0.2118 - categorical_accuracy: 0.4135 - fmeasure: 0.4184

KeyboardInterrupt: 

In [13]:
sequences = tokenizer.texts_to_sequences(munTexts)

munTextsCNN = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
prediction = modelCNN.predict(munTextsCNN)

for threshold in [.2,.25,.3,.35,.4,.45,.5,.6,.7,.8,.9]:
    realPrediction = (np.array(prediction) > threshold).astype(int)
    evaluation(yMun,realPrediction)
    print()

### TEST MODELS

In [14]:
modelstotest = ["best_models\weighted_CNN_W2V_loaded_dynamic_multi_234", "best_models\weighted_CNN_W2V_loaded_dynamic_multi_345", "best_models\weighted_CNN_W2V_loaded_dynamic_multi_789", "best_models\weighted_CNN_W2V_loaded_dynamic_single_11", "best_models\weighted_CNN_W2V_loaded_dynamic_single_3", "best_models\weighted_CNN_W2V_loaded_dynamic_single_5", "best_models\weighted_CNN_W2V_loaded_dynamic_single_7", "best_models\weighted_CNN_W2V_loaded_static_multi_234", "best_models\weighted_CNN_W2V_own_dynamic_multi_234", "best_models\weighted_CNN_W2V_own_dynamic_multi_345", "best_models\weighted_CNN_W2V_own_dynamic_multi_789", "best_models\weighted_CNN_W2V_own_dynamic_single_11", "best_models\weighted_CNN_W2V_own_dynamic_single_3", "best_models\weighted_CNN_W2V_own_dynamic_single_5", "best_models\weighted_CNN_W2V_own_dynamic_single_7"]
for modeltotest in modelstotest:
    oldModelCNN= keras.models.load_model(modeltotest, custom_objects={'fmeasure': fmeasure})
    prediction = oldModelCNN.predict(munTextsCNN)
    for threshold in [.2,.25,.3,.35,.4,.45,.5,.6,.7,.8,.9]:
        evaluation(yMun, (prediction >threshold).astype(int))
    print()

C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


F1 is 0.23, Precision is 0.134, Recall is 0.79, Accuracy is 0.021
F1 is 0.26, Precision is 0.159, Recall is 0.701, Accuracy is 0.03
F1 is 0.288, Precision is 0.19, Recall is 0.599, Accuracy is 0.036
F1 is 0.338, Precision is 0.26, Recall is 0.483, Accuracy is 0.077
F1 is 0.384, Precision is 0.365, Recall is 0.404, Accuracy is 0.144
F1 is 0.387, Precision is 0.494, Recall is 0.319, Accuracy is 0.289
F1 is 0.365, Precision is 0.592, Recall is 0.263, Accuracy is 0.301
F1 is 0.277, Precision is 0.723, Recall is 0.171, Accuracy is 0.273
F1 is 0.218, Precision is 0.824, Recall is 0.126, Accuracy is 0.248
F1 is 0.147, Precision is 0.857, Recall is 0.081, Accuracy is 0.221
F1 is 0.052, Precision is 0.842, Recall is 0.027, Accuracy is 0.18

F1 is 0.242, Precision is 0.144, Recall is 0.775, Accuracy is 0.016
F1 is 0.261, Precision is 0.163, Recall is 0.653, Accuracy is 0.023
F1 is 0.292, Precision is 0.198, Recall is 0.555, Accuracy is 0.036
F1 is 0.297, Precision is 0.226, Recall is 0.433, Accu

C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 is 0.2, Precision is 0.112, Recall is 0.894, Accuracy is 0.005
F1 is 0.217, Precision is 0.127, Recall is 0.767, Accuracy is 0.007
F1 is 0.254, Precision is 0.16, Recall is 0.611, Accuracy is 0.014
F1 is 0.29, Precision is 0.217, Recall is 0.438, Accuracy is 0.034
F1 is 0.301, Precision is 0.295, Recall is 0.307, Accuracy is 0.114
F1 is 0.233, Precision is 0.383, Recall is 0.168, Accuracy is 0.187
F1 is 0.155, Precision is 0.478, Recall is 0.092, Accuracy is 0.187
F1 is 0.069, Precision is 0.564, Recall is 0.037, Accuracy is 0.178
F1 is 0.055, Precision is 0.944, Recall is 0.029, Accuracy is 0.178
F1 is 0.023, Precision is 0.875, Recall is 0.012, Accuracy is 0.166
F1 is 0.007, Precision is 1.0, Recall is 0.003, Accuracy is 0.159

F1 is 0.188, Precision is 0.105, Recall is 0.896, Accuracy is 0.005
F1 is 0.212, Precision is 0.123, Recall is 0.763, Accuracy is 0.005
F1 is 0.244, Precision is 0.155, Recall is 0.574, Accuracy is 0.009
F1 is 0.298, Precision is 0.234, Recall is 0.411, Acc